In [1]:
import pandas as pd
import sqlite3
import os
import string
import mysql.connector
from sqlalchemy import create_engine

# Define your MySQL connection parameters
mysql_db_config = {
    'user': 'root',
    'password': '123321',
    'host': 'localhost',
    'database': 'nickdatabase'
}

# Now you can use mysql_db_config to connect to the database
conn = mysql.connector.connect(**mysql_db_config)
cursor = conn.cursor()




# Create an SQLAlchemy engine
engine = create_engine('mysql+mysqlconnector://{user}:{password}@{host}/{database}'.format(
    user='root',
    password='123321',
    host='localhost',
    database='nickdatabase'
))



# Paths
Fair_database_database = r'C:\Users\hofong\Desktop\python\Python working station\Database\TDC_Fair_database.db'
folder = r'C:\Users\nickf\OneDrive\桌面\Python\SQL\Excel file to database'
csv_container = r'C:\Users\nickf\OneDrive\桌面\Python\SQL\Excel file to database\CSV Converter'

if not os.path.exists(csv_container):
    os.makedirs(csv_container)

# Connect to the MySQL database
conn = mysql.connector.connect(**mysql_db_config)
cursor = conn.cursor()


file_list=[]
file_names=[]

excel_dic = {}




for root, directory, files in os.walk(folder):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)

            xls = pd.ExcelFile(file_path)
            for page_name in xls.sheet_names:
            # Check if the sheet name contains specific keywords
                if any(name_key in page_name.lower() for name_key in ['summary', 'non', 'local']):
                    try:
                        df = pd.read_excel(file_path, sheet_name=page_name, skiprows=range(9))
                            # Modify column headers
                        revised_headers = [str(header).replace("*","_").replace('#','_').replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header in df.columns]
                        df.columns = revised_headers
                        
                        # Save the DataFrame as a CSV file
                        change_to_csv_format = os.path.join(csv_container, file.replace('.xlsx', f'_{page_name}.csv')).replace('#','_')
                        df.to_csv(change_to_csv_format, index=False, encoding='utf_8_sig')
                        print(f"Saved CSV file: {change_to_csv_format}")
                    except:
                        continue


        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            file_list.append(file_path)
            file_name = file.replace(" ","_")
            file_names.append(file_name)
            excel_dic = {file_name:file_list for file_name, file_list in zip(file_names, file_list)}

for file_name, excel_path in excel_dic.items():
    df = pd.read_csv(excel_path)
    sql_revised_header = df.columns
    preparing_header1 = '`'+'` TEXT, `'.join(sql_revised_header)+'` TEXT'
    preparing_header2 = preparing_header1.split(',')
    preparing_header3 =[]
    
    #change data type
    for header in preparing_header2:
        if 'No.' in header or '$' in header or 'Total' in header or 'Booth_Area' in header or 'Net' in header or 'Charges' in header or 'Calculated' in header or 'Branch_Number' in header or 'Net_Order' in header or 'C&E' in header or 'SME' in header or 'product_listing' in header or 'Extended_online' in header or 'booth' in header or 'Package' in header or 'Price' in header or 'Calculated' in header or 'Amount' in header or 'charges' in header or 'eFair_Catalogue' in header or 'Price' in header or 'Unnamed' in header or 'Diff' in header:
            header = header.replace('TEXT', 'INTEGER')
            preparing_header3.append(header)
        elif 'Certificate_No.' in header or 'Certificate' in header:
            header = header.replace('INTEGER', 'TEXT PRIMARY KEY')
            preparing_header3.append(header)
        else:
            preparing_header3.append(header)
    cursor.execute (f'CREATE TABLE IF NOT EXISTS `{file_name}` ( {' ,'.join(preparing_header3)});')
    df.to_sql(file_name, engine, if_exists='append', index=False)
    conn.commit
    


    print(f"Successfully imported {file_name} into Database!")


conn.close

Successfully imported brand.csv into Database!
Successfully imported t_alibaba_data.csv into Database!
Successfully imported user.csv into Database!
Successfully imported user_brand.csv into Database!


<bound method CMySQLConnection.close of <mysql.connector.connection_cext.CMySQLConnection object at 0x000001B48B50FCE0>>